# Import packages

In [7]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans  
#from sklearn.inspection import silhouette_score
from sklearn.metrics import silhouette_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler  
 

# Feature select

In [8]:
df = pd.read_csv("./Data/CICIDOS2019_seclect_pingheng_72.csv")
df.shape

(220257, 72)

In [9]:
df

,Unnamed: 0.1,Unnamed: 0,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Inbound,Label
0,42237,20625,709,23454,17,1,2,0,458.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,4
1,78415,20520,793,35789,17,1,2,0,494.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,4
2,104140,190294,712,21344,17,1,2,0,422.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,4
3,113835,92325,895,51783,17,1,2,0,458.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,4
4,57410,99572,648,52824,17,48,2,0,458.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220252,318791,238369,34219,48349,6,116,2,2,12.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,5
220253,620523,401963,61272,36971,6,158,2,2,12.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,5
220254,12028,110759,35352,9698,17,989457,20,0,7242.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6
220255,86678,82330,51481,55083,17,2,2,0,766.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6


In [10]:
df['Label'].value_counts()

5    30199
6    29952
7    29932
0    29363
3    27618
2    26681
4    24044
1    22468
Name: Label, dtype: int64

### 使用特征间的PCCs进行谱聚类

In [ ]:
import pandas as pd
from sklearn.cluster import SpectralClustering
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt


pccs = df.corr()

# 转换PCCs矩阵为相似性矩阵
similarity_matrix = 1 - np.abs(pccs)

# 定义聚类数量范围
cluster_range = range(2, 15)

# 初始化一个列表来存储每个聚类数量的轮廓系数
silhouette_scores = []

# 遍历不同的聚类数量
for n_clusters in cluster_range:
    # 应用谱聚类
    spectral_cluster = SpectralClustering(n_clusters=n_clusters, affinity='precomputed', random_state=42)
    cluster_labels = spectral_cluster.fit_predict(similarity_matrix)

    # 计算轮廓系数
    silhouette_avg = silhouette_score(similarity_matrix, cluster_labels)
    silhouette_scores.append(silhouette_avg)

# 绘制轮廓系数随聚类数量变化的图
plt.figure(figsize=(10, 6))
plt.plot(cluster_range, silhouette_scores, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Coefficient')
plt.title('Silhouette Coefficient for Different Numbers of Clusters')
plt.show()

In [ ]:
import pandas as pd
from sklearn.cluster import SpectralClustering
from sklearn.metrics import silhouette_score

pccs = df.corr(method='pearson')

# 转换PCCs矩阵为相似性矩阵（使用1减去绝对值的PCCs）
similarity_matrix = 1 - pccs.abs()

# 假设我们想将特征分成k个聚类
k = 11  # 这个值需要根据你的具体数据进行调整

# 应用谱聚类
spectral_clust = SpectralClustering(n_clusters=k, affinity='precomputed')
clusters = spectral_clust.fit_predict(similarity_matrix)

# 计算轮廓系数
silhouette_avg = silhouette_score(similarity_matrix, clusters)
print("Silhouette Coefficient: ", silhouette_avg)

# 创建一个字典来保存每个聚类的特征名称和PCCs分数
clustered_features = {i: [] for i in range(k)}
clustered_pccs = {i: [] for i in range(k)}

# 分配特征和对应的PCCs分数到相应的聚类
for feature, cluster in zip(pccs.columns, clusters):
    clustered_features[cluster].append(feature)
    clustered_pccs[cluster].extend(pccs[feature])

# 打印每个聚类的特征数量和平均PCCs分数
for cluster in clustered_features:
    print("Cluster", cluster, "contains", len(clustered_features[cluster]), "features")
    avg_pccs = sum(clustered_pccs[cluster]) / len(clustered_pccs[cluster])
    print("Average PCCs score for Cluster", cluster, ":", avg_pccs)


### 标准化

In [ ]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

# 保存目标列 'class'
target = df['class']

# 选择需要标准化的特征列
features_to_scale = df.drop(columns=['class'])

# 初始化标准化器
scaler = StandardScaler()

# 对特征进行标准化
scaled_features = scaler.fit_transform(features_to_scale)

# 将标准化后的特征转换回DataFrame
df = pd.DataFrame(scaled_features, columns=features_to_scale.columns)

# 将目标列 'class' 添加回DataFrame
df['class'] = target

df


In [ ]:
from sklearn.cluster import Birch
from scipy.stats import pearsonr
import numpy as np
import pandas as pd



# 假设 'class' 是目标列
target = df['class']
features = df.drop(columns=['class'])

# 计算每个特征与目标变量的皮尔逊相关系数
correlation_scores = []
for col in features.columns:
    corr, _ = pearsonr(features[col], target)
    correlation_scores.append(corr)

# 将相关性分数转换为numpy数组
correlation_array = np.array(correlation_scores).reshape(-1, 1)

# 应用BIRCH聚类
birch = Birch(threshold=0.2, branching_factor=200, n_clusters=5)
clusters = birch.fit_predict(correlation_array)

# 初始化存储每个聚类的特征和平均PCCs分数的字典
clustered_features = {}
for cluster_num in range(max(clusters) + 1):
    # 获取当前聚类的特征
    cluster_indices = np.where(clusters == cluster_num)[0]
    cluster_feature_names = features.columns[cluster_indices]

    # 计算当前聚类的平均PCCs分数
    average_pccs_score = np.mean([correlation_scores[i] for i in cluster_indices])

    # 存储结果
    clustered_features[cluster_num] = {
        'features': list(cluster_feature_names),
        'average_pccs_score': average_pccs_score,
        'number_of_features': len(cluster_feature_names)
    }

# 输出每个聚类包含的特征数量、特征名称及其平均PCCs分数
for cluster_num, info in clustered_features.items():
    print(f"Cluster {cluster_num}: Number of Features: {info['number_of_features']}, Features: {info['features']}, Average PCCs Score: {info['average_pccs_score']}")


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from sklearn.cluster import Birch, SpectralClustering
from sklearn.tree import DecisionTreeClassifier

target = df['class']
features = df.drop('class', axis=1)

# 步骤1: 计算特征和目标列的皮尔逊相关系数
correlations = np.array([pearsonr(features[col], target)[0] for col in features.columns])

# 步骤2: 使用BIRCH进行初步聚类
birch = Birch(threshold=0.2, branching_factor=200, n_clusters=5)
birch_clusters = birch.fit_predict(correlations.reshape(-1, 1))

# 步骤3: 在每个BIRCH聚类上应用谱聚类进行细致分类
spectral_clusters = np.zeros_like(birch_clusters)
for cluster in np.unique(birch_clusters):
    cluster_indices = np.where(birch_clusters == cluster)[0]
    if len(cluster_indices) > 1:
        spectral = SpectralClustering(n_clusters=11)  # 调整细分聚类数量
        sub_clusters = spectral.fit_predict(correlations[cluster_indices].reshape(-1, 1))
        spectral_clusters[cluster_indices] = sub_clusters + np.max(spectral_clusters) + 1
    else:
        spectral_clusters[cluster_indices] = np.max(spectral_clusters) + 1

# 步骤4: 在每个聚类中使用决策树选择最优特征
selected_features = []
for cluster in np.unique(spectral_clusters):
    cluster_features = features.columns[np.where(spectral_clusters == cluster)[0]]
    if len(cluster_features) > 1:
        dt = DecisionTreeClassifier()
        dt.fit(features[cluster_features], target)
        importances = dt.feature_importances_
        selected_features.append(cluster_features[np.argmax(importances)])
    elif len(cluster_features) == 1:
        selected_features.append(cluster_features[0])

# 输出选择的最优特征
print("Selected features:", selected_features)


### 多分段聚类

In [ ]:
from scipy.stats import pearsonr
import pandas as pd

target = df['Label']
features = df.drop('Label', axis=1)

# 计算每个特征与目标变量的皮尔逊相关系数
correlation_scores = {}
for col in features.columns:
    corr, _ = pearsonr(features[col], target)
    correlation_scores[col] = corr

# 输出特征名称和对应的PCCs分数
for feature, score in correlation_scores.items():
    print(f"{feature}: {score}")


Unnamed: 0.1: -0.17540256641553426
Unnamed: 0: 0.4670222128640241
Source Port: 0.32521994403883
Destination Port: 0.15120080750960355
Protocol: 0.1517574827686086
Flow Duration: -0.0840553368294362
Total Fwd Packets: -0.004823482142125567
Total Backward Packets: -0.10931234950218764
Total Length of Fwd Packets: -0.17595775707459094
Total Length of Bwd Packets: -0.12012343871369437
Fwd Packet Length Max: -0.2976296141356495
Fwd Packet Length Min: -0.24267529248470326
Fwd Packet Length Mean: -0.2557830778589864
Fwd Packet Length Std: -0.1554558410977704
Bwd Packet Length Max: -0.1890011110206141
Bwd Packet Length Min: -0.1907477313675566
Bwd Packet Length Mean: -0.22837441231467
Bwd Packet Length Std: -0.19453910986699208
Flow Bytes/s: -0.3541641094595629
Flow Packets/s: -0.1525503071173295
Flow IAT Mean: -0.01610820622232109
Flow IAT Std: -0.06355001451846784
Flow IAT Max: -0.11458615440982904
Flow IAT Min: -0.06485590665633215
Fwd IAT Total: -0.07949340450685172
Fwd IAT Mean: -0.028484

In [13]:
from sklearn.cluster import Birch
import numpy as np

# 将PCCs分数转换为numpy数组以供BIRCH算法使用
correlation_array = np.array(list(correlation_scores.values())).reshape(-1, 1)

birch = Birch(threshold=0.2, branching_factor=100, n_clusters=None)
clusters = birch.fit_predict(correlation_array)

# 统计每个聚类的特征数量和特征名称
clustered_features = {}
for cluster_num in range(np.max(clusters) + 1):
    cluster_indices = np.where(clusters == cluster_num)[0]
    cluster_pccs_scores = [list(correlation_scores.values())[i] for i in cluster_indices]
    average_pccs_score = np.mean(cluster_pccs_scores) if cluster_pccs_scores else 0

    clustered_features[cluster_num] = {
        'number_of_features': len(cluster_indices),
        'feature_names': [list(correlation_scores.keys())[i] for i in cluster_indices],
        'average_PCCs_score': average_pccs_score
    }

# 输出每个聚类的特征数量、特征名称及平均PCCs分数
for cluster, info in clustered_features.items():
    print(f"Cluster {cluster}: Number of Features: {info['number_of_features']}, Feature Names: {info['feature_names']}, Average PCCs Score: {info['average_PCCs_score']}")

Cluster 0: Number of Features: 62, Feature Names: ['Unnamed: 0.1', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'SYN Flag Count', 'RST Flag Count', 'URG Flag Count', 'CWE Flag Count', 'Down/Up Ratio', 'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size', 'Subflo

### 谱聚类

In [31]:
from sklearn.cluster import SpectralClustering

# 使用谱聚类在BIRCH聚类的基础上进行更细致的划分
spectral_clustered_features = {}
for birch_cluster in range(np.max(clusters) + 1):
    # 选择当前BIRCH聚类中的特征
    birch_cluster_indices = np.where(clusters == birch_cluster)[0]
    birch_cluster_features = np.array(list(correlation_scores.values()))[birch_cluster_indices]
    birch_cluster_feature_names = np.array(list(correlation_scores.keys()))[birch_cluster_indices]

    # 动态决定谱聚类的簇数，确保至少有一个簇
    n_clusters = max(1, min(2, len(birch_cluster_indices) // 30))  

    # 检查是否有足够的数据点进行谱聚类
    if len(birch_cluster_indices) > 1:
        # 使用谱聚类进行进一步划分
        spectral = SpectralClustering(n_clusters=n_clusters, affinity='rbf')
        cluster_ids = spectral.fit_predict(birch_cluster_features.reshape(-1, 1))

        # 检查是否有足够的标签来计算轮廓系数
        if len(set(cluster_ids)) > 1:
            # 后处理：计算轮廓系数
            silhouette_avg = silhouette_score(birch_cluster_features.reshape(-1, 1), cluster_ids)
        else:
            silhouette_avg = None  # 或者设置为默认值
    else:
        # 当只有一个数据点时，无需进行聚类
        cluster_ids = [0]
        silhouette_avg = None  # 或者设置为默认值

    # 统计每个谱聚类的特征
    for cluster_id in np.unique(cluster_ids):
        cluster_indices = np.where(cluster_ids == cluster_id)[0]
        cluster_feature_names = birch_cluster_feature_names[cluster_indices]
        cluster_mean_score = np.mean(birch_cluster_features[cluster_indices])

        spectral_cluster_key = f"BIRCH Cluster {birch_cluster}, Spectral Cluster {cluster_id}"
        spectral_clustered_features[spectral_cluster_key] = {
            'number_of_features': len(cluster_indices),
            'feature_names': list(cluster_feature_names),
            'average_PCCs_score': cluster_mean_score,
            'silhouette_score': silhouette_avg  # 这里可以是 None 或者默认值
        }

# 输出谱聚类的结果
for key, value in spectral_clustered_features.items():
    print(f"{key}: {value}")

# 对谱聚类的结果按平均PCCs分数排序，并选择分数最高的前2类
top_clusters = sorted(spectral_clustered_features.items(), key=lambda x: abs(x[1]['average_PCCs_score']), reverse=True)[:3]
# 创建一个列表来保存所有最佳特征
best_features = []

# 输出选择的最佳特征类并添加到列表中
for key, value in top_clusters:
    print(f"Top Cluster {key}: Number of Features: {value['number_of_features']}, Feature Names: {value['feature_names']}, Average PCCs Score: {value['average_PCCs_score']}")
    best_features.extend(value['feature_names'])


BIRCH Cluster 0, Spectral Cluster 0: {'number_of_features': 22, 'feature_names': ['Unnamed: 0.1', 'Total Length of Fwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'URG Flag Count', 'CWE Flag Count', 'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size', 'Subflow Fwd Bytes', 'Init_Win_bytes_backward'], 'average_PCCs_score': -0.23852716570396176, 'silhouette_score': 0.5829105237111878}
BIRCH Cluster 0, Spectral Cluster 1: {'number_of_features': 40, 'feature_names': ['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Std', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT 

### 选择①类9个特征

In [17]:
# 加入原始数据集中的目标列（假设目标列名为 'class'）
best_features.append('Label')
df01 = df[best_features]
df01

,Unnamed: 0,Source Port,Destination Port,Protocol,Fwd Header Length,ACK Flag Count,Fwd Header Length.1,min_seg_size_forward,Inbound,Label
0,20625,709,23454,17,16,0,16,8,1,4
1,20520,793,35789,17,40,0,40,20,1,4
2,190294,712,21344,17,16,0,16,8,1,4
3,92325,895,51783,17,16,0,16,8,1,4
4,99572,648,52824,17,33,0,33,16,1,4
...,...,...,...,...,...,...,...,...,...,...
220252,238369,34219,48349,6,40,1,40,20,1,5
220253,401963,61272,36971,6,40,1,40,20,1,5
220254,110759,35352,9698,17,160,0,160,8,1,6
220255,82330,51481,55083,17,64,0,64,32,1,6


In [18]:
df01.to_csv("./Data/pccs_julei_9.csv")

### 选择②类22个特征

In [19]:
import pandas as pd

feature_names = ['Unnamed: 0.1', 'Total Length of Fwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'URG Flag Count', 'CWE Flag Count', 'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size', 'Subflow Fwd Bytes', 'Init_Win_bytes_backward','Label']
df02 = df[feature_names]
df02

,Unnamed: 0.1,Total Length of Fwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,...,Packet Length Std,Packet Length Variance,URG Flag Count,CWE Flag Count,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Subflow Fwd Bytes,Init_Win_bytes_backward,Label
0,42237,458.0,229.0,229.0,229.0,0.0,0.0,0.0,0.0,4.580000e+08,...,0.000000,0.000000,0,0,343.50,229.0,0.0,458,-1,4
1,78415,494.0,247.0,247.0,247.0,0.0,0.0,0.0,0.0,4.940000e+08,...,0.000000,0.000000,0,0,370.50,247.0,0.0,494,-1,4
2,104140,422.0,211.0,211.0,211.0,0.0,0.0,0.0,0.0,4.220000e+08,...,0.000000,0.000000,0,0,316.50,211.0,0.0,422,-1,4
3,113835,458.0,229.0,229.0,229.0,0.0,0.0,0.0,0.0,4.580000e+08,...,0.000000,0.000000,0,0,343.50,229.0,0.0,458,-1,4
4,57410,458.0,229.0,229.0,229.0,0.0,0.0,0.0,0.0,9.541667e+06,...,0.000000,0.000000,0,0,343.50,229.0,0.0,458,-1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220252,318791,12.0,6.0,6.0,6.0,6.0,6.0,6.0,0.0,2.068966e+05,...,0.000000,0.000000,0,0,7.50,6.0,6.0,12,0,5
220253,620523,12.0,6.0,6.0,6.0,6.0,6.0,6.0,0.0,1.518987e+05,...,0.000000,0.000000,0,0,7.50,6.0,6.0,12,0,5
220254,12028,7242.0,401.0,321.0,362.1,0.0,0.0,0.0,0.0,7.319166e+03,...,30.703885,942.728571,0,0,378.15,362.1,0.0,7242,-1,6
220255,86678,766.0,383.0,383.0,383.0,0.0,0.0,0.0,0.0,3.830000e+08,...,0.000000,0.000000,0,0,574.50,383.0,0.0,766,-1,6


In [20]:
df02.to_csv("./Data/pccs_julei_22.csv")

### 选择③类特征40个

In [21]:
import pandas as pd

feature_names = ['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Std', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'SYN Flag Count', 'RST Flag Count', 'Down/Up Ratio', 'Subflow Fwd Packets', 'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward', 'act_data_pkt_fwd', 'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min','Label']
df03 = df[feature_names]
df03

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Bwd Packets,Fwd Packet Length Std,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,...,act_data_pkt_fwd,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,1,2,0,0.0,0.000000,2.000000e+06,1.000000,0.000000,1.0,1.0,...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
1,1,2,0,0.0,0.000000,2.000000e+06,1.000000,0.000000,1.0,1.0,...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
2,1,2,0,0.0,0.000000,2.000000e+06,1.000000,0.000000,1.0,1.0,...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
3,1,2,0,0.0,0.000000,2.000000e+06,1.000000,0.000000,1.0,1.0,...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
4,48,2,0,0.0,0.000000,4.166667e+04,48.000000,0.000000,48.0,48.0,...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220252,116,2,2,12.0,0.000000,3.448276e+04,38.666667,65.240580,114.0,1.0,...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
220253,158,2,2,12.0,0.000000,2.531646e+04,52.666667,54.151023,109.0,1.0,...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
220254,989457,20,0,0.0,30.127623,2.021311e+01,52076.684211,56393.837202,110771.0,0.0,...,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
220255,2,2,0,0.0,0.000000,1.000000e+06,2.000000,0.000000,2.0,2.0,...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6


In [22]:
df03.to_csv("./Data/pccs_julei_40.csv")

### 选择①②类 31个特征

In [29]:
# 加入原始数据集中的目标列（假设目标列名为 'class'）
best_features.append('Label')
df04 = df[best_features]
df04

,Unnamed: 0.1,Total Length of Fwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,...,Unnamed: 0,Source Port,Destination Port,Protocol,Fwd Header Length,ACK Flag Count,Fwd Header Length.1,min_seg_size_forward,Inbound,Label
0,42237,458.0,229.0,229.0,229.0,0.0,0.0,0.0,0.0,4.580000e+08,...,20625,709,23454,17,16,0,16,8,1,4
1,78415,494.0,247.0,247.0,247.0,0.0,0.0,0.0,0.0,4.940000e+08,...,20520,793,35789,17,40,0,40,20,1,4
2,104140,422.0,211.0,211.0,211.0,0.0,0.0,0.0,0.0,4.220000e+08,...,190294,712,21344,17,16,0,16,8,1,4
3,113835,458.0,229.0,229.0,229.0,0.0,0.0,0.0,0.0,4.580000e+08,...,92325,895,51783,17,16,0,16,8,1,4
4,57410,458.0,229.0,229.0,229.0,0.0,0.0,0.0,0.0,9.541667e+06,...,99572,648,52824,17,33,0,33,16,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220252,318791,12.0,6.0,6.0,6.0,6.0,6.0,6.0,0.0,2.068966e+05,...,238369,34219,48349,6,40,1,40,20,1,5
220253,620523,12.0,6.0,6.0,6.0,6.0,6.0,6.0,0.0,1.518987e+05,...,401963,61272,36971,6,40,1,40,20,1,5
220254,12028,7242.0,401.0,321.0,362.1,0.0,0.0,0.0,0.0,7.319166e+03,...,110759,35352,9698,17,160,0,160,8,1,6
220255,86678,766.0,383.0,383.0,383.0,0.0,0.0,0.0,0.0,3.830000e+08,...,82330,51481,55083,17,64,0,64,32,1,6


In [30]:
df04.to_csv("./Data/pccs_julei_31.csv")

### 选择①②③类 71个特征

In [32]:
# 加入原始数据集中的目标列（假设目标列名为 'class'）
best_features.append('Label')
df05 = df[best_features]
df05

,Unnamed: 0.1,Total Length of Fwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,...,act_data_pkt_fwd,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,42237,458.0,229.0,229.0,229.0,0.0,0.0,0.0,0.0,4.580000e+08,...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
1,78415,494.0,247.0,247.0,247.0,0.0,0.0,0.0,0.0,4.940000e+08,...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
2,104140,422.0,211.0,211.0,211.0,0.0,0.0,0.0,0.0,4.220000e+08,...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
3,113835,458.0,229.0,229.0,229.0,0.0,0.0,0.0,0.0,4.580000e+08,...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
4,57410,458.0,229.0,229.0,229.0,0.0,0.0,0.0,0.0,9.541667e+06,...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220252,318791,12.0,6.0,6.0,6.0,6.0,6.0,6.0,0.0,2.068966e+05,...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
220253,620523,12.0,6.0,6.0,6.0,6.0,6.0,6.0,0.0,1.518987e+05,...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
220254,12028,7242.0,401.0,321.0,362.1,0.0,0.0,0.0,0.0,7.319166e+03,...,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
220255,86678,766.0,383.0,383.0,383.0,0.0,0.0,0.0,0.0,3.830000e+08,...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6


In [33]:
df05.to_csv("./Data/pccs_julei_71.csv")